In [ ]:
!pip install ultralytics flask-ngrok flask pillow

# 🧠 Load YOLOv8 and define vehicle counting function
from ultralytics import YOLO
from PIL import Image
from flask import Flask, request, jsonify
from flask_ngrok import run_with_ngrok
import io

In [ ]:

# Load pre-trained YOLOv8 model (you can replace with custom one if fine-tuned later)
model = YOLO('yolov8x.pt')  # You can also use 'yolov8m.pt' or 'yolov8n.pt' for faster speed
model.fuse()  # Optimize the model

In [ ]:
# Define the class IDs for vehicle types: car, motorcycle, bus, truck
vehicle_class_ids = [2, 3, 5, 7]  # COCO dataset classes

In [ ]:
# Vehicle counting logic
def count_vehicles(image: Image.Image):
    results = model.predict(image, verbose=False)[0]
    count = sum(1 for cls in results.boxes.cls if int(cls) in vehicle_class_ids)
    return count


In [ ]:
# 🔌 Set up Flask server with Ngrok tunnel
app = Flask(__name__)
run_with_ngrok(app)  # This creates a public URL for the Colab-hosted API

@app.route('/count', methods=['POST'])
def count_route():
    if 'image' not in request.files:
        return jsonify({'error': 'No image uploaded'}), 400

    img_file = request.files['image']
    image = Image.open(img_file.stream).convert("RGB")  # Ensure consistent format
    count = count_vehicles(image)
    return jsonify({'vehicle_count': count})

# 🚀 Start the Flask app
print("Starting vehicle counting API server...")
app.run()